In [1]:
import json
import  pickle
import joblib
import optuna
import os
import time
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy  as np
import pandas as pd
import tqdm
import time

from sklearn.cluster import KMeans
from eval_utils import *
from models_gaussian import GaussianDenseHMM, HMMLoggingMonitor, DenseHMMLoggingMonitor
from models_gaussian_A import GaussianDenseHMM as CoocHMM
from hmmlearn import hmm

2022-08-22 06:28:00.505249: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-22 06:28:00.505315: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
t = time.localtime()
RESULT_DIR = f'gaussian_dense_hmm_benchmark/eval-cooc-{t.tm_year}-{t.tm_mon}-{t.tm_mday}'

We have examined the efficiency of co-occurence based learning methods for Gaussian HMM. This notebook contains a comparison of:

- standard Gaussian Hidden Markov Model implementation,
- co-occurrence based larning for Gaussian Hidden Markov Model,
- EM learning for GaussianDenseHHMM,
- co-occurrence based learning for GaussianDenseHMM.

The dense representation was examined with fixed and tuned embedding length.

In [3]:
n = 10
s = 100
T = 2000
simple_model = True
l_fixed = True

In [4]:
# RESULT_DIR = f'gaussian_dense_hmm_benchmark/eval-cooc-{t.tm_year}-{t.tm_mon}-{17}'
# with open(f"{RESULT_DIR}/optuna_cooc_s{s}_T{T}_n{n}_simple_model{simple_model}_l{l_fixed}.pkl", "rb") as f:
#     study = joblib.load(f)

In [5]:
# study_res = pd.DataFrame([{**t.params, "mean": t.values[0], "std": t.values[1]} for t in study.trials])
# study_res

In [6]:
# sns.scatterplot(data=study_res, x="mean", y="std")

In [7]:
# best_params = dict()
# # best_params["cooc"] = study_res.sort_values("mean").iloc[-1, :2].to_dict()
# best_params["cooc"] = study_res.sort_values("std").iloc[1, :2].to_dict()
# best_params["cooc"]['cooc_epochs_param'] = int(best_params["cooc"]['cooc_epochs_param'])
# best_params["cooc"]['l_param'] = int(np.ceil(n / 3))
# best_params

In [8]:
np.random.seed(2022)

best_result = {}
s, T, n, pi, A, mu, sigma, result, true_values, wandb_params, X_true, Y_true, lengths, _, em_scheduler = init_experiment((s, T, n), simple_model)
nodes = np.concatenate([np.array([-np.infty, Y_true.min()]),
                        (mu[1:] + mu[:-1]) / 2,
                        np.array([Y_true.max(), np.infty])])
m = nodes.shape[0] - 1

models = dict(cooc=CoocHMM, dense=GaussianDenseHMM, dense_em=GaussianDenseHMM)
monitors = dict(cooc=DenseHMMLoggingMonitor, dense=DenseHMMLoggingMonitor, dense_em=DenseHMMLoggingMonitor)
algs = dict(cooc="cooc", dense="cooc", dense_em="em")

# ## Tune hyper-parameters
# l = np.ceil(n / 3) if l_fixed else None

In [9]:
# lengths = np.array(lengths)

In [10]:
# def _lengths_iterator(seqs, lengths):
#     n_seqs = len(lengths)
#     left, right = 0, 0
#
#     for i in range(len(lengths)):
#         right += lengths[i]
#         yield seqs[left:right]
#         left += lengths[i]

In [11]:
params = {'cooc_lr_param': 0.2200833144117315, 'cooc_epochs_param': 997953, 'l_param': 4}

In [ ]:
## Evaluate models

true_values = {
    "states": X_true,
    "transmat": A,
    "startprob": pi,
    "means": mu,
    "covars": sigma
}

# HMMlearn
best_result["HMMlearn"] = list()
wandb_params["init"].update({"job_type": f"n={n}-s={s}-T={s}-simple={simple_model}",
                             "name": f"HMMlearn"})
wandb_params["config"].update(dict(model="HMMlearn", m=0, l=0, lr=0,
                                   em_iter=em_iter(n), cooc_epochs=0,
                                   epochs=0), scheduler=False, simple_model=simple_model)
nodes_tmp = mu
nodes = np.concatenate([np.array([-np.infty, Y_true.min()]),
                        (nodes_tmp[1:] + nodes_tmp[:-1]).reshape(-1) / 2,
                        np.array([Y_true.max(), np.infty])])
Y_disc = (Y_true > nodes[1:].reshape(1, -1)).sum(axis=-1).reshape(-1, 1)

# # for _ in range(10):
# for _ in range(8):
#     hmm_monitor = HMMLoggingMonitor(tol=TOLERANCE, n_iter=0, verbose=True,
#                                     wandb_log=True, wandb_params=wandb_params, true_vals=true_values,
#                                     log_config={'metrics_after_convergence': True})
#     hmm_model = hmm.GaussianHMM(n, n_iter=em_iter(n))
#     hmm_model.monitor_ = hmm_monitor
#     hmm_model.fit(Y_true, lengths)
#
#     preds = hmm_model.predict(Y_true, lengths)
#     perm = find_permutation(preds, X_true)
#
#     best_result["HMMlearn"].append(
#         {
#             "time": time.perf_counter() - hmm_monitor._init_time,
#             "logprob": hmm_model.score(Y_true, lengths),
#             "acc": (X_true == np.array([perm[i] for i in preds])).mean(),
#             "dtv_transmat": dtv(hmm_model.transmat_, A[perm, :][:, perm]),
#             "dtv_startprob": dtv(hmm_model.startprob_, pi[perm]),
#             "MAE_means": (abs(mu[perm] - hmm_model.means_[:, 0])).mean(),
#             "MAE_sigma": (abs(sigma.reshape(-1)[perm] - hmm_model.covars_.reshape(-1))).mean(),
#             "dtv_omega": dtv(empirical_cooc_prob(Y_disc, n+2, lengths),
#                              normal_cooc_prob(hmm_model.means_.reshape(-1), hmm_model.covars_.reshape(-1), nodes[1:], A))
#         }
#     )

# Custom models
# for name in tqdm.tqdm(["cooc", "dense", "dense_em"],  desc="Model building"):
for name in tqdm.tqdm(["dense", "dense_em"],  desc="Model building"):
    model = models[name]
    monitor = monitors[name]
    alg = algs[name]
    # params = best_params[name]
    best_result[name] = list()
    wandb_params["init"].update({"job_type": f"n={n}-s={s}-T={s}-simple={simple_model}",
                                 "name": f"{name}-l={params['l_param']}-lr={params['cooc_lr_param']}-epochs={params['cooc_epochs_param']}"})
    wandb_params["config"].update(dict(model="dense_cooc", m=0, l=int(params['l_param']), lr=params['cooc_lr_param'],
                                       em_iter=em_iter(n), cooc_epochs=params['cooc_epochs_param'],
                                       epochs=params['cooc_epochs_param']), scheduler=True,
                                  simple_model=simple_model)

    # for _ in tqdm.tqdm(range(10), desc=f"Training {name}"):
    for _ in tqdm.tqdm(range(8), desc=f"Training {name}"):
        hmm_monitor = monitor(tol=TOLERANCE, n_iter=0, verbose=True,
                              wandb_log=True, wandb_params=wandb_params, true_vals=true_values,
                              log_config={'metrics_after_convergence': True})
        # kmeans = KMeans(n_clusters=n, random_state=0).fit(Y_true)
        densehmm = model(n, mstep_config={'cooc_epochs': params['cooc_epochs_param'],
                                          'cooc_lr': params['cooc_lr_param'],
                                          "l_uz": int(params['l_param']),
                                          'loss_type': 'square',
                                          'scheduler': em_scheduler},
                         covariance_type='diag', logging_monitor=hmm_monitor, nodes=nodes,
                         init_params="", params="stmc", early_stopping=False, opt_schemes={"cooc"},
                         discrete_observables=m)
        if alg == "cooc":
            densehmm.fit_coocs(Y_true, lengths)
        else:
            densehmm.fit(Y_true, lengths)

        preds = densehmm.predict(Y_true, lengths)
        perm = find_permutation(preds, X_true)

        best_result[name].append(
            {
                "time": time.perf_counter() - hmm_monitor._init_time,
                "logprob": densehmm.score(Y_true, lengths),
                "acc": (X_true == np.array([perm[i] for i in preds])).mean(),
                "dtv_transmat": dtv(densehmm.transmat_, A[perm, :][:, perm]),
                "dtv_startprob": dtv(densehmm.startprob_, pi[perm]),
                "MAE_means": (abs(mu[perm] - densehmm.means_[:, 0])).mean(),
                "MAE_sigma": (abs(sigma.reshape(-1)[perm] - densehmm.covars_.reshape(-1))).mean(),
                "dtv_omega": dtv(empirical_cooc_prob(Y_disc, n+2, lengths),
                                  normal_cooc_prob(densehmm.means_.reshape(-1), densehmm.covars_.reshape(-1), nodes[1:], A))
            }
        )

with open(f"{RESULT_DIR}/good_result_s{s}_T{T}_n{n}_simple_model{simple_model}_l{l_fixed}.json", "w") as f:
    json.dump(best_result, f, indent=4)

Training dense:   0%|                                                                                                                                     | 0/8 [00:00<?, ?it/s]

accuracy,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
startprob_dtv,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
time,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
total_log_prob,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
transmat_dtv,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
accuracy,0.54055
covars_mae,nan
means_mae,nan
omage_dtv,nan
startprob_dtv,0.07376
time,481.98995


         1     -194020.1399             +nan
         2     -190068.6668       +3951.4731
         3     -190561.9174        -493.2506
         4     -190714.5815        -152.6641
         5     -190741.0806         -26.4991
         6     -190728.3638         +12.7168
         7     -190701.3010         +27.0628
         8     -190668.6647         +32.6363
         9     -190634.3410         +34.3236
        10     -190600.2385         +34.1026
        11     -190567.2985         +32.9400
        12     -190535.9637         +31.3348
        13     -190506.4222         +29.5415
        14     -190478.7364         +27.6858
        15     -190452.9091         +25.8273
        16     -190428.9150         +23.9941
        17     -190406.7152         +22.1998
        18     -190386.2638         +20.4514
        19     -190367.5100         +18.7538
        20     -190350.3997         +17.1103
        21     -190334.8755         +15.5242
        22     -190320.8778         +13.9978
        23

accuracy,▁▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇▇████
covars_mae,▇███▇▇▇▇▆▆▆▆▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▁▁▁▁
means_mae,▅▆▆▅▅▄▄▃▃▂▂▂▁▁▁▂▂▂▃▃▃▄▄▄▄▅▅▅▆▆▆▆▆▇▇▇▇███
omage_dtv,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
startprob_dtv,▁▆█████████████████████████▇▇▇▇▇▇▇▇▇▇▇▇▇
time,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
total_log_prob,▇███▇▇▇▇▆▆▆▆▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▁▁▁▁
transmat_dtv,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
accuracy,0.8559
covars_mae,0.60739
means_mae,0.00855


         1     -144155.0501             +nan
         2     -120874.1812      +23280.8690
         3     -120714.3640        +159.8172
         4     -120655.3643         +58.9997
         5     -120600.9209         +54.4434
         6     -120678.9162         -77.9953
         7     -120783.0166        -104.1004
         8     -120837.5584         -54.5419
         9     -120862.5719         -25.0135
        10     -120872.4245          -9.8525
        11     -120874.4321          -2.0076
        12     -120872.3963          +2.0358
        13     -120868.3468          +4.0495
        14     -120863.3721          +4.9747
        15     -120858.0473          +5.3249
        16     -120852.6667          +5.3806
        17     -120847.3741          +5.2925
        18     -120842.2348          +5.1393
        19     -120837.2744          +4.9604
        20     -120832.4996          +4.7747
        21     -120827.9090          +4.5906
        22     -120823.4973          +4.4117
        23

...

         1      -68858.1306             +nan
         2      -58517.0131      +10341.1175
         3      -58351.1452        +165.8679
         4      -58289.5841         +61.5611
         5      -58248.2679         +41.3162
         6      -58219.6719         +28.5959
         7      -58200.3001         +19.3719
         8      -58186.8243         +13.4757
         9      -58176.9155          +9.9088
        10      -58169.1562          +7.7593
        11      -58162.7259          +6.4303
        12      -58157.1567          +5.5692
        13      -58152.1826          +4.9741
        14      -58147.6518          +4.5308
        15      -58143.4770          +4.1748
        16      -58139.6070          +3.8700
        17      -58136.0104          +3.5966
        18      -58132.6665          +3.3439
        19      -58129.5601          +3.1063
        20      -58126.6787          +2.8814
        21      -58124.0106          +2.6681
        22      -58121.5441          +2.4665
        23

accuracy,▁▂▄▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇▇█████████████████████
covars_mae,█▆▅▅▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁
means_mae,▃▁▂▃▄▄▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇████████████
omage_dtv,█▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
startprob_dtv,▁███████████████████████████████████████
time,▁▁▁▁▁▁▁▂▂▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▆▆▆▆▆▇▇▇▇██
total_log_prob,▁▅▅▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇██████████████████████
transmat_dtv,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
accuracy,0.9977
covars_mae,0.49155
means_mae,0.04493


         1      -64614.6424             +nan
         2      -57230.6549       +7383.9875
         3      -56903.3615        +327.2934
         4      -56869.2463         +34.1152
         5      -56875.8731          -6.6267
         6      -56886.4403         -10.5672
         7      -56894.5730          -8.1327
         8      -56899.3701          -4.7971
         9      -56901.2115          -1.8414
        10      -56900.7337          +0.4778
        11      -56898.5399          +2.1938
        12      -56895.1255          +3.4143
        13      -56890.8731          +4.2524
        14      -56886.0678          +4.8054
        15      -56880.9170          +5.1508
        16      -56875.5689          +5.3481
        17      -56870.1275          +5.4414
        18      -56864.6644          +5.4631
        19      -56859.2280          +5.4364
        20      -56853.8500          +5.3780
        21      -56848.5505          +5.2995
        22      -56843.3414          +5.2091
        23

accuracy,▁▂▂▂▃▃▄▄▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇███████████
covars_mae,█▇▆▆▆▆▅▄▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁
means_mae,▁▂▃▄▅▅▆▆▇▇▇▇▇▇▇█████████████████████████
omage_dtv,█▆▅▅▄▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
startprob_dtv,█▄▃▄▄▄▄▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
time,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
total_log_prob,▁▂▃▄▅▅▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇████████████████
transmat_dtv,██▇▅▄▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
accuracy,0.99665
covars_mae,0.78307
means_mae,0.13168


         1     -177511.0505             +nan
         2     -159055.6698      +18455.3807
         3     -161066.5510       -2010.8812
         4     -162239.2051       -1172.6541
         5     -163193.1203        -953.9152
         6     -164040.2857        -847.1654
         7     -164808.0178        -767.7322
         8     -165508.8595        -700.8417
         9     -166152.2076        -643.3481
        10     -166745.8915        -593.6839
        11     -167296.1472        -550.2556
        12     -167807.8544        -511.7072
        13     -168285.0794        -477.2251
        14     -168731.4629        -446.3834
        15     -169150.3128        -418.8499
        16     -169544.5652        -394.2524
        17     -169916.7501        -372.1849
        18     -170269.0050        -352.2549
        19     -170603.1268        -334.1217
        20     -170920.6380        -317.5113
        21     -171222.8477        -302.2097
        22     -171510.8970        -288.0493
        23

In [ ]:
from sklearn.decomposition import PCA

z_init = np.transpose(hmm_monitor.z[-1])
pca_z = PCA(n_components=2).fit(z_init)
z = [pca_z.transform(np.transpose(x)) for x in hmm_monitor.z] + [pca_z.transform(z_init)]

z0 = list(hmm_monitor.z0)

u_init = hmm_monitor.u[-1]
pca_u = PCA(n_components=2).fit(u_init)
u = [pca_u.transform(x) for x in hmm_monitor.u] + [pca_u.transform(u_init)]

draw_embeddings(z, "z")
draw_embeddings(z0, "z0")
draw_embeddings(u, "u")